# Perceptual-Hashing


Here we'll have a look at the perceptual hashing, as well as a little bit of testing/bug hunting on the already implemented image manipulation methods.

NOTE: If running locally on `mps` can only load one model at a time.

In [ ]:
# load image of me
from diffusers.utils import load_image

from arc_phash.image_manipulation.utils import resize_and_crop

me_img = load_image("../images/me_2.jpg")
resize_and_crop(me_img)

In [ ]:
# load inpainter
from pprint import pprint

from arc_phash.image_manipulation.deepfake import img2img

inpainter = img2img.Img2Img(device="mps", resize_size=512)
pprint(inpainter.pipe_kwargs)

In [ ]:
import numpy as np

strengths = np.linspace(0.1, 0.9, 5)

for strength in strengths:
    inpainter.generate_img2img(image=me_img, strength=strength)

In [7]:
out_imgs = inpainter.get_results()

## Perceptual Hashes

Now it appears that the image generation methods work, lets start comapring some perceptual hashes of them.

In [7]:
import numpy as np
import pdqhash

hash_vec_orig, qual_o = pdqhash.compute(np.asarray(me_img))
hash_vec_alt, qual_a = pdqhash.compute(np.asarray(out_img))

In [ ]:
from scipy.spatial.distance import euclidean, hamming, sqeuclidean

print("distances from original to original:")
print(f"sqeuclidean: {sqeuclidean(hash_vec_orig, hash_vec_orig)}")
print(f"euclidean: {euclidean(hash_vec_orig, hash_vec_orig)}")
print(f"hamming: {hamming(hash_vec_orig, hash_vec_orig)}")

print("distances from original to inpainted:")
print(f"sqeuclidean: {sqeuclidean(hash_vec_orig, hash_vec_alt)}")
print(f"euclidean: {euclidean(hash_vec_orig, hash_vec_alt)}")
print(f"hamming: {hamming(hash_vec_orig, hash_vec_alt)}")

In [13]:
import pdqhash


def hash_dist(img_1, img_2):
    """compute hash distance based on pdq hash"""
    hash_1, _ = pdqhash.compute(np.asarray(img_1))
    hash_2, _ = pdqhash.compute(np.asarray(img_2))
    return sqeuclidean(hash_1, hash_2)

In [20]:
import matplotlib.pyplot as plt
from scipy.spatial.distance import sqeuclidean


def plotter(imgs: list):
    """plot images and distances"""
    fig, ax = plt.subplots(1, len(imgs), figsize=(20, 15))
    _ = [ax[idx].imshow(img) for idx, img in enumerate(imgs)]
    _ = [
        ax[idx].set_title(f"{hash_dist(imgs[0], img)}") for idx, img in enumerate(imgs)
    ]
    _ = [axe.axis("off") for axe in ax]
    fig.suptitle("Squared euqlidean distance between PHash of original and infill")
    plt.show()

In [ ]:
images = []
images.append(out_imgs[0]["original"])
for out in out_imgs:
    images.append(out["img2img"])

plotter(images)

In [ ]:
out_imgs